# Test notebook for evaluation of Measuring Up privacy leakage analysis

In [1]:
from lxml import html
import requests
import urllib2
from bs4 import BeautifulSoup
import csv
from urlparse import urlsplit, urlunsplit
import datetime

In [2]:
# Define Variables

# Limited test of the partial collection of libraries in the CSV file
limitedTest = True


# Get Member Library Names and URLs
dataDLF = 'https://www.diglib.org/members/'
dataARL = 'http://www.arl.org/membership/list-of-arl-members'
dataOCLC = 'http://www.oclc.org/research/partnership/roster.html'

# data file containing the list of libraries from which to retrieve pages
libraryFile = "data_all_library.csv"

# root path for writing out the retrieved HTML and Javascript files
writePath = "fileStore"
logPath = "analysisLogs"

In [3]:
# Connect to the CSV file containing the list of libraries to test
with open(libraryFile, 'rb') as csvfile:
    libraryReader = csv.DictReader(csvfile, delimiter=",", quotechar='"')
    i=0
    for libraryRow in libraryReader:
        i+=1
        print "(" + str(i) + ") " + "Processing: " + libraryRow['school']
        
        # break the provided URL into components so that secure and insecure requests can be generated
        scheme,host,path,query,fragment = urlsplit(libraryRow['url'])
        insecureLink = urlunsplit(('http',host,path,query,fragment))
        secureLink = urlunsplit(('https',host,path,query,fragment))
        print "Insecure URL: " + insecureLink
        print "Secure URL: " + secureLink
        
        # request the insecure page and process its content
        try:
            insecurePageRequest = requests.get(insecureLink, allow_redirects=True)
            pageHeaders = insecurePageRequest.headers
            pageCookies = insecurePageRequest.cookies
            pageRedirectHist = insecurePageRequest.history
            pageRedirectEndURL = insecurePageRequest.url
            pageHTML = insecurePageRequest.content
            if urlsplit(pageRedirectEndURL)[0] == 'https':
                pageSecureRedirect = True
            else:
                pageSecureRedirect = False
            print pageHeaders
            print pageCookies
            print pageRedirectHist
            print pageRedirectEndURL
            print pageSecureRedirect
            thisFileWritePath = writePath + "/" + host
            thisFileWriteFilename = "insecure_" + host + ".html"
            ### need to add code to created a subdirectory for placing the files
            f = open(writePath+'/'+thisFileWriteFilename, 'w')
            f.write(pageHTML)
            f.close()
        except:
            print "something went wrong - more soon ;)"
        
         # request the secure page and process its content
        try:
            securePageRequest = requests.get(secureLink, allow_redirects=True)
            pageHeaders = securePageRequest.headers
            pageCookies = securePageRequest.cookies
            pageRedirectHist = securePageRequest.history
            pageRedirectEndURL = securePageRequest.url
            pageHTML = securePageRequest.content
            if urlsplit(pageRedirectEndURL)[0] == 'https':
                pageSecureRedirect = True
            else:
                pageSecureRedirect = False
            print pageHeaders
            print pageCookies
            print pageRedirectHist
            print pageRedirectEndURL
            print pageSecureRedirect
            thisFileWritePath = writePath + "/" + host
            thisFileWriteFilename = "secure_" + host + ".html"
            ### need to add code to created a subdirectory for placing the files
            f = open(writePath+'/'+thisFileWriteFilename, 'w')
            f.write(pageHTML)
            f.close()
        except:
            print "something went wrong - more soon ;)"
        

        print
        print "======================================"
        if limitedTest and i>10:
            break

(1) Processing: University of Alabama Libraries
Insecure URL: http://www.lib.ua.edu/
Secure URL: https://www.lib.ua.edu/
{'x-powered-by': 'PHP/5.3.17', 'transfer-encoding': 'chunked', 'vary': 'Accept-Encoding', 'keep-alive': 'timeout=15, max=100', 'server': 'Apache/2.2.12 (Linux/SUSE)', 'connection': 'Keep-Alive', 'link': '<https://www.lib.ua.edu/wp-json/>; rel="https://api.w.org/", <https://www.lib.ua.edu/>; rel=shortlink', 'date': 'Wed, 13 Jul 2016 13:58:08 GMT', 'content-type': 'text/html; charset=UTF-8', 'x-pingback': 'https://www.lib.ua.edu/xmlrpc.php'}
<RequestsCookieJar[]>
[<Response [301]>]
https://www.lib.ua.edu/
True
{'x-powered-by': 'PHP/5.3.17', 'transfer-encoding': 'chunked', 'vary': 'Accept-Encoding', 'keep-alive': 'timeout=15, max=100', 'server': 'Apache/2.2.12 (Linux/SUSE)', 'connection': 'Keep-Alive', 'link': '<https://www.lib.ua.edu/wp-json/>; rel="https://api.w.org/", <https://www.lib.ua.edu/>; rel=shortlink', 'date': 'Wed, 13 Jul 2016 13:58:09 GMT', 'content-type': 